In [3]:
import requests
import os
import json

# URL to fetch the JSON data
url = "https://csrc.nist.gov/extensions/nudp/services/json/nudp/framework/version/sp_800_171_3_0_0/export/json?element=all"

# File path to store the JSON data
framework_file_path = "../src/api/data/sp_800_171_3_0_0/framework.json"

# Fetch the JSON data
response = requests.get(url)
framework_data = response.json()

# Ensure the directory exists
os.makedirs(os.path.dirname(framework_file_path), exist_ok=True)

# Write the JSON data to the file
with open(framework_file_path, 'w') as file:
    json.dump(framework_data, file, indent=4)

print(f"JSON data has been stored in {framework_file_path}")

JSON data has been stored in ../src/api/data/sp_800_171_3_0_0/framework.json


In [4]:
import re
import time

# Regex match form 03.14.01 and 03.14
pattern = re.compile(r'\d{2}\.\d{2}(?:\.\d{2})?')

for element in framework_data["response"]["elements"]["elements"]:
    identifier = element['element_identifier']
    if pattern.match(identifier):

        url = f"https://csrc.nist.gov/extensions/nudp/services/json/nudp/framework/version/sp_800_171_3_0_0/element/{identifier}/graph"
        file_path = (
            f"../src/api/data/sp_800_171_3_0_0/elements/{identifier}.json"
        )

        # Fetch the JSON data
        response = requests.get(url).json()

        # Ensure the directory exists
        os.makedirs(os.path.dirname(file_path), exist_ok=True)

        with open(file_path, "w") as file:
            json.dump(response, file, indent=4)


        # Add sleep to prevent throttling
        time.sleep(1)

